<a href="https://colab.research.google.com/github/udlbook/udlbook/blob/main/Trees/SAT_Construction_Answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SAT Constructions

The purpose of this Python notebook is to investigate SAT constructions that impose constraints on sets of variables.  We'll build constructions for ensuring all of the variables are the same,  that only one of the variables is true, that at leats $K$ of the variables are true, that fewer than $K$ of the variables are true and that exactly $K$ of the variables are true.

You can save a local copy of this notebook in your Google account and work through it in Colab (recommended) or you can download the notebook and run it locally using Jupyter notebook or similar.

Contact me at iclimbtreesmail@gmail.com if you find any mistakes or have any suggestions.

In [ ]:
!pip install z3-solver
from z3 import *
import numpy as np
from itertools import combinations

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 20.4 MB/s eta 0:00:00


# Same

Let's write a subroutine that returns a Boolean logic formula that constrains $N$ variables to be the same.

$$ \textrm{Same}[x_{1},x_{2},x_{3},
\ldots x_{N}]:= (x_{1}\land x_{2}\land x_{3},\ldots,x_N)\lor(\overline{x}_{1}\land \overline{x}_{2}\land \overline{x}_{3},\ldots,\overline{x}_N).$$

First let's define the variables.  We'll choose $N=10$

In [ ]:
N=10
x = [ z3.Bool("x_{i}".format(i=i)) for i in range(0,N) ]
print(x)

Now let's write the main routine.  We can make use of the fact that the Z3 command 'Or' which combines together a list of literals with Or operations.

In [ ]:
# Routine that returns the SAT construction (a Boolean logic formula) for all literals in x being the same.
def same(x):
  return Or(And(x), And([Not(xi) for xi in x]))

Finally, let's test that our routine works.  Here's generic routine that evaluates a vector of concrete values against a provided Boolean logic formula


In [ ]:
def check_expression(x, formula, literals):
  # Create the solver
  s = Solver()
  # Add the constraint
  s.add(formula(x))
  # Add the literals
  s.add(And([x[i] if literal else Not(x[i]) for i, literal in enumerate(literals)]))
  # Check if it's SAT (creates the model)
  sat_result = s.check()
  if(sat_result==sat):
    print("True")
  else:
    print("False")

In [ ]:
check_expression(x, same, [False,False,False,False,False,False,False,False,False,False])
check_expression(x, same, [True,False,False,False,False,False,False,False,False,False])
check_expression(x, same, [False,False,False,True,False,False,True,False,True,False])
check_expression(x, same, [True,True,True,True,True,True,True,True,True,True])

# Exactly One

Now let's write a subroutine that returns a Boolean logic formula that is true when only one of the provided variables are the same.

First, we write a formula that ensures at least one of the variables is true:

$$\textrm{AtLeastOne}[x_1,x_2,x_3]:= x_{1}\lor x_{2} \lor x_{3}, \ldots, x_{N}.$$

Then, we write a formula that ensures that no more than one is true.  For each possible pair of literals, we ensure that they are not both true:

$$\textrm{NoMoreThanOne}[x_{1},x_{2},x_{3}]:=  \lnot (x_{1}\land x_{2}) \land \lnot (x_{1}\land x_{3}),\ldots, \land \lnot (x_{n-1}\land x_{N}) $$

Finally, we **AND** together these two formulae:

$$\textrm{ExactlyOne}[x_1,x_2,x_3,\ldots, x_N] = \textrm{AtLeastOne}[x_1,x_2,x_3, \ldots, x_N]\land\textrm{NoMoreThanOne}[x_1,x_2,x_3, \ldots x_N]$$

You might want to use the function "combinations" from itertools (imported above).  Example usage:

In [ ]:
test = [ z3.Bool("x_{i}".format(i=i)) for i in range(0,4) ]
for comb in combinations(test, 2):
  print(comb[0], comb[1])

In [ ]:
def at_least_one(x):
  return Or(x)

def no_more_than_one(x):
    clauses = True ;
    for comb in combinations(x, 2):
      clauses = And([clauses, Or([Not(comb[0]), Not(comb[1])]) ])
    return clauses

def exactly_one(x):
    return And(at_least_one(x), no_more_than_one(x))

Let's test if this works:

In [ ]:
check_expression(x, exactly_one, [False,False,False,False,False,False,False,False,False,False])
check_expression(x, exactly_one, [True,False,False,False,False,False,False,False,False,False])
check_expression(x, exactly_one, [False,False,False,True,False,False,True,False,True,False])
check_expression(x, exactly_one, [True,True,True,True,True,True,True,True,True,True])

# At least K

Finally, we'll build the construction that ensures that there at at least K elements that are true in a vector of length N.

This one is a bit more complicated.  We construct an NxK matrix of new literals  $r_{i,j}$  Then we add the following constraints.

1. Top left element $r_{0,0}$ is the first data element

$$ r_{00} \Leftrightarrow x_{1}$$

2. Remaining elements $r_{0,1:}$ must be false

$$ \overline{r}_{0,j} \quad\quad \forall j\in\{1,2,\ldots K-1\}$$

3. Remaining elements $r_{i,0}$ in first column are true where x_i is true

$$x_i \Rightarrow r_{i,0} \quad\quad \forall i\in\{1,2,\ldots N-1\}$$

4. For rows 1:N-1,  if the element above is true, this must be true

$$ r_{i-1,j} \Rightarrow r_{i,j}\quad\quad \quad\quad \forall i\in\{1,2,\ldots N-1\}, j\in\{0,1,\ldots,K-1\}$$

5. If x is true for this row and the element above and to the left is true then set this element to true.

$$ (x_i \land r_{i-1,j-1})\Rightarrow r_{i,j} \quad\quad \forall i\in\{1,2,\ldots N-1\}, j\in\{1,2,\ldots,K-1\} $$

6. If x is false for this row and the element above is false then set to false

$$ (\overline{x}_{i} \land \overline{r}_{i-1,j}) \Rightarrow \overline{r}_{i,j} \quad\quad \forall i\in\{1,2,\ldots N-1\}, j\in\{1,2,\ldots,K-1\} $$

7. if x is false for this row and the element above and to the left is false then set to false:

$$ (\overline{x}_i \land \overline{r}_{i-1,j-1})\Rightarrow \overline{r}_{i,j} \quad\quad \forall i\in\{1,2,\ldots N-1\}, j\in\{1,2,\ldots,K-1\} $$

8.  The bottom-right element of r must be true

$$ r_{N-1,K-1}$$

In [ ]:
def at_least_k(x, K):
  # Create nxk table of literals r_{i,j}
  N = len(x) ;
  r = [[ z3.Bool("r_{%d,%d}"%((i,j))) for j in range(0,K) ] for i in range(0,N) ]

  #1. Top left element $r_{0,0}$ is the first data element

  clauses = (r[0][0] == x[0])

  #2. Remaining elements $r_{0,1:}$ must be false

  clauses = And(clauses, And([ Not(r[0][j]) for j in range(1,K) ]))

  #3. Remaining elements $r_{i,0}$ in first column are true where x_i is true
  clauses = And(clauses, And([Implies(x[i],r[i][0]) for i in range(1,N)]))

  #4. For rows 1:n-1,  if the element above is true, this must be true

  for i in range(1,N):
    for j in range(0,K):
      clauses = And(clauses, Implies(r[i-1][j], r[i][j]))

  #5. If x is true for this row and the element above and to the left is true then set this element to true.
  #6. If x is false for this row and the element above is false then set to false
  #7. if x is false for this row and the element above and to the left is false then set to false:
  for i in range(1,N):
    for j in range(1,K):
      clauses = And(clauses, Implies(And(x[i], r[i-1][j-1]), r[i][j]))
      clauses = And(clauses, Implies(And(Not(x[i]), Not(r[i-1][j])), Not(r[i][j])))
      clauses = And(clauses, Implies(And(Not(x[i]), Not(r[i-1][j-1])), Not(r[i][j])))

  #8. The bottom-right element of r must be true
  clauses = And(clauses, r[N-1][K-1])

  return clauses

In [ ]:
#This function defined for convenience so that check_expression works
def at_least_three(x):
  return at_least_k(x,3)

check_expression(x, at_least_three, [False,False,False,False,False,False,False,False,False,False])
check_expression(x, at_least_three, [True,False,False,False,False,False,False,False,False,False])
check_expression(x, at_least_three, [False,False,False,True,False,False,True,False,True,False])
check_expression(x, at_least_three, [True,True,True,True,True,True,True,True,True,True])

Optional:  write functions to test if there are:

*   Fewer than $K$ elements
*   Exactly $N$ elements



In [ ]:
def fewer_than_k(x, K):
  # Create nxk table of literals r_{i,j}
  N = len(x) ;
  r = [[ z3.Bool("r_{%d,%d}"%((i,j))) for j in range(0,K) ] for i in range(0,N) ]

  #1. Top left element $r_{0,0}$ is the first data element

  clauses = (r[0][0] == x[0])

  #2. Remaining elements $r_{0,1:}$ must be false

  clauses = And(clauses, And([ Not(r[0][j]) for j in range(1,K) ]))

  #3. Remaining elements $r_{i,0}$ in first column are true where x_i is true
  clauses = And(clauses, And([Implies(x[i],r[i][0]) for i in range(1,N)]))

  #4. For rows 1:n-1,  if the element above is true, this must be true
  for i in range(1,N):
    for j in range(0,K):
      clauses = And(clauses, Implies(r[i-1][j], r[i][j]))

  #5. If x is true for this row and the element above and to the left is true then set this element to true.
  #6. If x is false for this row and the element above is false then set to false
  #7. if x is false for this row and the element above and to the left is false then set to false:
  for i in range(1,N):
    for j in range(1,K):
      clauses = And(clauses, Implies(And(x[i], r[i-1][j-1]), r[i][j]))
      clauses = And(clauses, Implies(And(Not(x[i]), Not(r[i-1][j])), Not(r[i][j])))
      clauses = And(clauses, Implies(And(Not(x[i]), Not(r[i-1][j-1])), Not(r[i][j])))

  #8. The bottom-right element of r must be false
  clauses = And(clauses, Not(r[N-1][K-1]))

  return clauses

In [ ]:
#This function defined for convenience so that check_expression works
def fewer_than_three(x):
  return fewer_than_k(x,3)

check_expression(x, fewer_than_three, [False,False,False,False,False,False,False,False,False,False])
check_expression(x, fewer_than_three, [True,False,False,False,False,False,False,False,False,False])
check_expression(x, fewer_than_three, [False,False,False,True,False,False,True,False,True,False])
check_expression(x, fewer_than_three, [True,True,True,True,True,True,True,True,True,True])

In [ ]:
def exactly_k(x, K):
  # Create n x (k+1) table of literals r_{i,j}
  N = len(x) ;
  r = [[ z3.Bool("r_{%d,%d}"%((i,j))) for j in range(0,K+1) ] for i in range(0,N) ]

  #1. Top left element $r_{0,0}$ is the first data element

  clauses = (r[0][0] == x[0])

  #2. Remaining elements $r_{0,1:}$ must be false

  clauses = And(clauses, And([ Not(r[0][j]) for j in range(1,K+1) ]))

  #3. Remaining elements $r_{i,0}$ in first column are true where x_i is true
  clauses = And(clauses, And([Implies(x[i],r[i][0]) for i in range(1,N)]))

  #4. For rows 1:n-1,  if the element above is true, this must be true
  for i in range(1,N):
    for j in range(0,K+1):
      clauses = And(clauses, Implies(r[i-1][j], r[i][j]))

  #5. If x is true for this row and the element above and to the left is true then set this element to true.
  #6. If x is false for this row and the element above is false then set to false
  #7. if x is false for this row and the element above and to the left is false then set to false:
  for i in range(1,N):
    for j in range(1,K+1):
      clauses = And(clauses, Implies(And(x[i], r[i-1][j-1]), r[i][j]))
      clauses = And(clauses, Implies(And(Not(x[i]), Not(r[i-1][j])), Not(r[i][j])))
      clauses = And(clauses, Implies(And(Not(x[i]), Not(r[i-1][j-1])), Not(r[i][j])))

  #8. The bottom-right element of r must be
  clauses = And(clauses, r[N-1][K-1])
  clauses = And(clauses, Not(r[N-1][K]))

  return clauses

In [ ]:
#This function defined for convenience so that check_expression works
def exactly_three(x):
  return exactly_k(x,3)

check_expression(x, exactly_three, [False,False,False,False,False,False,False,False,False,False])
check_expression(x, exactly_three, [True,False,False,False,False,False,False,False,False,False])
check_expression(x, exactly_three, [False,False,False,True,False,False,True,False,True,False])
check_expression(x, exactly_three, [True,True,True,True,True,True,True,True,True,True])

Now that we've finished, I'll let you into a secret.  Z3 actually contains routines for (AtLeast, FewerThanOrEqualToo, and Exactly)

In [ ]:
def at_least_k_z3(x,K):
  return PbGe([(i,1) for i in x],K)

def at_least_three_z3(x):
  return at_least_k_z3(x,3)

check_expression(x, at_least_three_z3, [False,False,False,False,False,False,False,False,False,False])
check_expression(x, at_least_three_z3, [True,False,False,False,False,False,False,False,False,False])
check_expression(x, at_least_three_z3, [False,False,False,True,False,False,True,False,True,False])
check_expression(x, at_least_three_z3, [True,True,True,True,True,True,True,True,True,True])

In [ ]:
def fewer_than_k_z3(x,K):
  return PbLe([(i,1) for i in x],K-1)

def fewer_than_three_z3(x):
  return fewer_than_k_z3(x,3)

check_expression(x, fewer_than_three_z3, [False,False,False,False,False,False,False,False,False,False])
check_expression(x, fewer_than_three_z3, [True,False,False,False,False,False,False,False,False,False])
check_expression(x, fewer_than_three_z3, [False,False,False,True,False,False,True,False,True,False])
check_expression(x, fewer_than_three_z3, [True,True,True,True,True,True,True,True,True,True])

In [ ]:
def exactly_k_z3(x,K):
  return PbEq([(i,1) for i in x],K)

def exactly_three_z3(x):
  return exactly_k_z3(x,3)

check_expression(x, exactly_three_z3, [False,False,False,False,False,False,False,False,False,False])
check_expression(x, exactly_three_z3, [True,False,False,False,False,False,False,False,False,False])
check_expression(x, exactly_three_z3, [False,False,False,True,False,False,True,False,True,False])
check_expression(x, exactly_three_z3, [True,True,True,True,True,True,True,True,True,True])